In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#MNIST class prediction

mnist는 숫자를 손으로 그린 그림에 대한 데이터  
각 그림을 보고 어떠한 숫자가 적혀있는지 classification 하는 딥러닝 모델 구축  
multi class classification은 강의자료의 appendix 참조  

주요 모듈 목록  
**torch.utils.data.Dataset:** 데이터셋을 getitem method에서 한개씩 텐서형태로 반환하도록 구성하는 class  
**torch.utils.data.DataLoader:** batch processing을 위해 dataset에서 반환되는 데이터를 n개의 batch_size만큼 묶어서 반환하도록 하는 class  
**torch.nn.Module:** 딥러닝 모델을 구현한 class, forward method를 통해 입력된 텐서 데이터를 딥러닝 연산하여 결과 반환  
**torch.nn.CrossEntropyLoss:** loss 연산을 위한 class이며 backward method를 통해 gradient계산  
**torch.optim.Adam:** 계산된 gradient를 update해주어 feadient descent를 진행하는 class  

# Dataset class구현  
  
dataset(fashion-mnist) : https://www.kaggle.com/datasets/zalando-research/fashionmnist?resource=download

target: label(적힌 숫자)  
input feature: pixel1 ~ pixel 784까지의 명도
train/test file이 구분되어있음  

각 data를 torch.utils.data.Dataset을 통해 하나씩 load할 수 있는 class구현  
이후 torch.utils.data.DataLoader를 통해 batch개씩 변환

torch.utils.data.Dataset의 주요 method
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(데이터셋, 데이터경로)등을 생성  
__ len __: 해당 클래스에서 다루는 dataset의 길이를 반환하는 함수  
__ getitem __(index): index에 해당하는 데이터 하나를 tensor형태로 반환하는 함수

In [2]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

train_data_path = "/content/drive/MyDrive/nlp-open-tutorial/2일차_배포/dataset/fashion-mnist_train.csv"
test_data_path = "/content/drive/MyDrive/nlp-open-tutorial/2일차_배포/dataset/fashion-mnist_test.csv"
batch_size = 64

#torch.utils.data.Dataset을 상속하여 Dataset class선언
class myDataset(Dataset):
  #오브젝트를 선언할때 불러오는 함수, superclass(부모클래스)의 init을 실행해 주어야함
  def __init__(self, df_data) -> None:
    super().__init__()
    self.y = df_data.loc[:,["label"]]
    self.x = df_data.drop(["label"], axis=1)

  #list 형태의 class를 만들때 필수로 사용되는 함수, 전체 길이를 알아야 인덱싱이 가능
  def __len__(self):
    return len(self.y)

  #index에 해당하는 데이터를 반환해주는 함수
  def __getitem__(self, index):
    data = torch.Tensor(self.x.loc[index,:])
    target = torch.LongTensor(self.y.loc[index,:])
    return data, target

#data load후 분할
train_data_df = pd.read_csv(train_data_path).dropna().loc[:5000,:]
test_data_df = pd.read_csv(test_data_path).dropna().loc[:1000,:]


#train dataset, test dataset을 각각 선언
trainDataset = myDataset(train_data_df)
testDataset = myDataset(test_data_df)

#torch.utils.data.DataLoader을 사용 dataloader 선언, 이를 통해 batch processing
trainDataloader = DataLoader(trainDataset, batch_size)
testDataloader = DataLoader(testDataset, batch_size)

#잘 작동하는지 test
for i in trainDataset:
  print("dataset test")
  print(i)
  break

#잘 작동하는지 test
for i in trainDataloader:
  print("data loader test")
  data = i[0]
  target = i[1]

  print("data")
  print(data)
  print(data.shape)
  print("\ntarget")
  print(target)
  print(target.shape)
  break


dataset test
(tensor([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   4.,   0.,   0.,   0.,   0.,   0.,  62.,  61.,
         21.,  29.,  23.,  51., 136.,  61.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  88.,
        201., 228., 225., 255., 115.,  62., 137., 255., 235., 222., 255., 135.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,  47., 252., 

# Deep Learning Model 구현  

10개의 label의 확률을 예측해야하는 모델  
그렇기에 output layer의 최종 결과가 10차원의 vector가 되어야함

torch.nn.Module을 이용하여 모델 구현  
1st hidden layer의 feature는 10개  
2nd hidden layer의 feature는 100개인 모델을 구현한다.  

torch.nn.Linear: perceptron의 weighted sum과 같이 linaer regression연산을 하는 calss  
torch.nn.ReLU: ReLU activation function을 수행하는 class  

torch.nn.Module의 주요 method  
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(사용할 deep learning layer, activation function, 등)등을 생성  
__ forward __(data): 입력받은 data를 딥러닝 모델을 통해 결과를 예측하여 반환하는 class  

**각 딥러닝 레이어 연산 중 차원수를 확인하고 잘 맞춰줄 것**  
참고 document(해당 사이트의 shape를 확인하고 tensor형태 결정)  
nn.Linear: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html  
nn.ReLU: https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

In [3]:
from torch import nn

#딥러닝 모델을 작성하기위한 모듈
class myModel(nn.Module):
  #오브젝트를 선언할 때 불러와지는 함수 일반적으로 이번 모델에서 사용될 각 레이어들이 포함됨
  def __init__(self) -> None:
      super().__init__()
      self.linear = nn.Linear(784, 10)
      self.linear2 = nn.Linear(10, 100)
      self.linear3 = nn.Linear(100, 10)
      self.relu = nn.ReLU()

  #데이터를 입력받고 딥러닝 연산후 결과를 반환하는 함수
  def forward(self, x):
      x = self.linear(x)
      x = self.relu(x)
      x = self.linear2(x)
      x = self.relu(x)
      x = self.linear3(x)
      return x

#작성한 모델 선언
model = myModel()

#잘 작동하는지 test
for i in trainDataloader:
  print("model test")
  data = i[0]
  target = i[1]

  print("input data")
  print(data)
  print(data.shape)
  print("\noutput predict")
  print(model(data))
  print(model(data).shape)
  print("\nground thruth")
  print(target)
  print(target.shape)
  break


model test
input data
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([64, 784])

output predict
tensor([[ 1.0968e+00, -2.6694e+01, -9.5882e-01, -1.7280e+00,  8.5507e+00,
         -5.2811e+00,  4.4647e+00, -6.9400e+00, -1.7173e+01,  1.0489e+01],
        [-3.5807e+00, -5.9574e+00, -4.9482e+00,  2.5328e+00, -2.3187e+00,
         -4.7688e+00,  3.2444e+00, -3.3278e+00, -7.9297e+00,  1.1053e+01],
        [-3.9013e+00, -1.3315e+01, -5.8114e+00, -4.5361e+00, -3.3558e+00,
         -1.3393e+01,  1.5805e+01, -8.8826e+00, -1.8060e+01,  7.4419e+00],
        [-1.7865e+00,  5.2013e-02, -1.0224e+01,  5.3646e-01,  4.5045e+00,
         -2.5247e+00, -1.4849e+00,  4.1400e+00, -7.8903e+00,  1.5217e+01],
        [-1.1252e+00, -2.1223e+01, -7.6173e+00,  2.1989e+00,  6.8870e+00,
         -1.5786

# 작성한 dataset과 model을 이용하여 딥러닝 프로세스 구현  

pytorch 딥러닝 프로세스
1. dataset, model선언
2. dataset과 model을 통한 결과 예측
3. 예측된 결과를 통해 **loss**연산 및 **loss.backward**
4. **optimizer.step()**를 사용하여 graident update

주요 오브젝트  
torch.nn.CrossEntropyLoss: classification를 위한 loss 수행  
torch.optim.Adam: Adam optimizer를 통해 gradient update를 수행하는 class

각 오브젝트의 입력과 선언은 다음 doc 참조:  
CrossEntropyLoss: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html  
Adam: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html

In [6]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

#학습을 위한 optimizer와 loss function 설정
lf = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

#100번의 epoch을 실행
for e in range(100):
  print("\nepoch ", e)
  epoch_loss = 0

  #선언한 model을 학습 가능한 상태로 변경
  model.train()

  #모든 학습데이터에 대해서 학습
  for i in trainDataloader:
    #매 배치에 대한 gradient계산 이전에 optimizer에 저장된 이전 batch에 gradient를 삭제(초기화)
    optimizer.zero_grad()

    data = i[0]
    target = i[1]

    #결과 도출
    output = model(data)

    #loss연산
    loss =lf(output, target.squeeze())

    #loss backpropagation
    loss.backward()

    #gradient update
    optimizer.step()
    epoch_loss += loss.item()

  print("train loss", epoch_loss/len(trainDataloader))

  #model이 학습되지 않는 상태로 변경
  model.eval()
  test_loss = 0

  #gradient를 계산하지 않도록 하여 cost낭비 방지
  with torch.no_grad():
    #모든 test dataset에 대해서 결과연산
    for i in testDataloader:
      data = i[0]
      target = i[1]

      output = model(data)

      loss = lf(output, target.squeeze())
      test_loss += loss.item()

      pred_label = torch.argmax(output, dim=-1)
      # print(sum(pred_label == target.squeeze))

  print("test loss", test_loss/len(testDataloader))




epoch  0
train loss 1.7072110085547725
test loss 1.658403143286705

epoch  1
train loss 1.6092683303205273
test loss 1.6677372753620148

epoch  2
train loss 1.6100349260281912
test loss 1.6214602813124657

epoch  3
train loss 1.6019917168194735
test loss 1.7606471255421638

epoch  4
train loss 1.6038827956477297
test loss 1.5996231734752655

epoch  5
train loss 1.5984083987489532
test loss 1.59738277643919

epoch  6
train loss 1.6144927468480943
test loss 1.6437721848487854

epoch  7
train loss 1.6219052181968205
test loss 1.6222265809774399

epoch  8
train loss 1.590532340580904
test loss 1.6892364993691444

epoch  9
train loss 1.6138902464999427
test loss 1.8889990001916885

epoch  10
train loss 1.620734522614298
test loss 1.8029621690511703

epoch  11
train loss 1.6610238491734373
test loss 1.6321982890367508

epoch  12
train loss 1.6906937330583982
test loss 1.7462877109646797

epoch  13
train loss 1.9429145025301584
test loss 1.8707768842577934

epoch  14
train loss 1.65124246289